In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
import tensorflow as tf
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, brier_score_loss, roc_curve
from sklearn.calibration import calibration_curve
from sklearn.impute import KNNImputer

from keras.layers import Dense, Flatten
from keras import Sequential
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.models import load_model
from keras.utils import to_categorical
from keras import metrics

from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier, XGBRFClassifier

import os
import warnings
warnings.filterwarnings('ignore')

# mlflow.sklearn.autolog()
# import mlflow

## Выделение выборок

In [ ]:
# Загрузка данных

df_train = pd.read_csv('data/train_data_processed.csv')
df_test = pd.read_csv('data/test_data_processed.csv')

In [ ]:
# Выделение выборки валидации из тренировочной части

df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=42)

In [ ]:
# Сохраним валидационный датасет

df_valid.to_csv('data/valid_data_processed.csv')

In [ ]:
feature_columns = [
    'Client_age', 'Gender', 'Numb_of_Prod', 'Salary',
    'HasCrCard', 'Numb_of_years', 'CreditScore', 'Balance', 'IsActiveMember',
]
target_col = 'res'

### Параметры моделей

___Параметры для поиска оптимальной комбинации значений при обучении модели___

In [ ]:
params_xgb = {
    'n_estimators': np.arange(1, 100, 25),
    'max_depth': np.arange(1, 10, 3),
    'max_leaves': np.arange(0, 20, 3),
    # 'grow_policy': [0, 1],
    'learning_rate': np.arange(0.1, 1, 0.3),
    # 'booster': ['gbtree', 'gblinear', 'dart'],
    # 'subsample': np.arange(0.6, 0.8, 0.1),
    # 'colsample_bytree': np.arange(0.6, 0.8, 0.1),
    # 'colsample_bylevel': np.arange(0.6, 0.8, 0.1),
    # 'colsample_bynode': np.arange(0.6, 0.8, 0.1),
    'reg_alpha': np.arange(0, 1, 0.5),
    'reg_lambda': np.arange(0, 1, 0.5),
}

params_lgb = {
    'n_estimators': np.arange(1, 100, 25),
    'max_depth': np.arange(1, 10, 3),
    'num_leaves': np.arange(0, 20, 3),
    'learning_rate': np.arange(0.1, 1, 0.3),
    # 'boosting_type': ['gbdt', 'goss', 'dart'],
    # 'subsample': np.arange(0.6, 0.8, 0.1),
    # 'colsample_bytree': np.arange(0.6, 0.8, 0.1),
    'reg_alpha': np.arange(0, 1, 0.5),
    'reg_lambda': np.arange(0, 1, 0.5),
}

params_rf = {
    'n_estimators': np.arange(1, 100, 50),
    'max_leaves': np.arange(0, 20, 10),
    'learning_rate': np.arange(0.1, 1, 0.3),
    'reg_alpha': np.arange(0, 1, 0.5),
    'reg_lambda': np.arange(0, 1, 0.5),
}

params_log = {
    'l1_ratio': np.arange(0, 1, 0.2),
    'C': np.arange(0.001, 0.01, 0.001),
}